In [2]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [3]:
!python -V

Python 3.9.12


In [4]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Q1 

What's the standard deviation of the predicted duration for this dataset?

1.24
6.24
12.28
18.28

In [13]:
y_pred_series = pd.Series(y_pred)
std_deviation = y_pred_series.std()
print("Standard Deviation of Predictions:", std_deviation)

Standard Deviation of Predictions: 6.247489794200264


In [16]:
# Create an artificial ride_id column
year = 2023
month =  #March 2023
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
#Write the ride_id and the predictions to a dataframe with the results
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],  
    'prediction': y_pred
})

In [18]:
#Save it as a parquet

#path: https://mlops-bucket-mannerow.s3.us-east-1.amazonaws.com/

output_file = 'results'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)